In [38]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

In [39]:
cities_df = pd.read_csv('../WeatherPy/WeatherPy_output.csv', index_col = False)
cities_df.head()

,City ID,City,Country,Lng,Lat,Cloudiness,Humidity,Date,Max Temp,Wind Speed
0,2156643,Mount Gambier,AU,140.77,-37.83,90.0,72.0,"Saturday, 13. February 2021 04:06PM",57.470,4.12
1,3466750,Cassilândia,BR,-51.73,-19.11,100.0,85.0,"Saturday, 13. February 2021 04:06PM",74.696,0.81
2,2063056,Port Augusta,AU,137.77,-32.50,0.0,64.0,"Saturday, 13. February 2021 04:06PM",68.270,5.66
3,3517970,Pochutla,MX,-96.47,15.74,20.0,54.0,"Saturday, 13. February 2021 04:06PM",87.278,10.29
4,3366880,Hermanus,ZA,19.23,-34.42,0.0,88.0,"Saturday, 13. February 2021 04:06PM",68.270,2.85


In [40]:
# Humidity Heatmap
gmaps.configure(api_key=gkey)

# add 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)

In [41]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

fig.add_layer(heat_layer)

fig
# plt.savefig('output_data/heatmap_layer.png')

Figure(layout=FigureLayout(height='420px'))

In [42]:
# Create new DataFrame fitting weather criteria

# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
address =  ['Hermanus', 'Mexicali', 'Hilo']
hotel_df = cities_df[(cities_df['Max Temp'] > 70) & (cities_df['Max Temp'] < 80)& (cities_df['Wind Speed'] < 10)& (cities_df['Cloudiness'] < 3)]


hotel_df


,City ID,City,Country,Lng,Lat,Cloudiness,Humidity,Date,Max Temp,Wind Speed
20,2319257,Wamba,NG,8.6,8.93,0.0,18.0,"Saturday, 13. February 2021 04:06PM",77.648,1.79


In [52]:
# Hotel Map

# geocoordinates
target_coordinates = ['-34.42, 19.24','32.68, -115.50','19.71,-155.08']
# target_search = "ZA"
target_radius = 5000
target_type = "hotels"

# set up a parameters dictionary
params = {
    "location": target_coordinates[1],
#     "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request 
response = requests.get(base_url, params=params)

In [53]:
# convert response to json
places_data = response.json()

# Print the json 
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLXEZjSrwbZXdKlOv32J_UZ7gXF8rD78POcPR2E7mGfyl11GsSlHtesoqCcQjQ8Eq2p9jKIwwSl7dxaShbj6Bvp907eCNoQv2D4SzkMcVr7LyxvTxlFDqoHdWfhLle3quiC4ACwQHVKEDIQUfI-317mWhblUcqBQFKf7pD83Y_1l1PU437j7EeKdqND69BT4TTxuKTDttdgkKsZjtTKUo1_9Ccn2mG3qhDQ8tTl34NjkJOJs0oQiXISGyZBh80Ssie9FBJCF9MAc4SjxGWQqWO5e0w3sJhZQVOIWYL6c9VsRORvt6nepetm_ohVXzfVA9mfBGudK9SdaibDTtHdGKQFQJGyqMm0Q7fxvuXaVFm0XhM1Juc_GahgDxzqimiCLJc9JN_RfRo0QItXcJKCcwq_Lg163k3D5nSrt-3rhqy8XdYRS21Y0yk",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 32.6245389,
                    "lng": -115.4522623
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.67234628483124,
                        "lng": -115.3417968921595
                    },
                    "southwest": {
                        "lat": 32.55517080747436,
                        "lng": -115.558785072

In [54]:
print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])

Mexicali
Mexicali


In [64]:
# Store into variable named hotel_df
hotel_df = cities_df
hotel_df['Hotel Name'] = np.nan

target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

idx = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Lat']}, {row['Lng']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[idx, -1] = name
    idx = idx + 1

narrowed_city_df = hotel_df
narrowed_city_df.head()

,City ID,City,Country,Lng,Lat,Cloudiness,Humidity,Date,Max Temp,Wind Speed,Hotel Name
0,2156643,Mount Gambier,AU,140.77,-37.83,90.0,72.0,"Saturday, 13. February 2021 04:06PM",57.470,4.12,Commodore on the Park
1,3466750,Cassilândia,BR,-51.73,-19.11,100.0,85.0,"Saturday, 13. February 2021 04:06PM",74.696,0.81,Hotel Rafa Cassilândia
2,2063056,Port Augusta,AU,137.77,-32.50,0.0,64.0,"Saturday, 13. February 2021 04:06PM",68.270,5.66,Majestic Oasis Apartments
3,3517970,Pochutla,MX,-96.47,15.74,20.0,54.0,"Saturday, 13. February 2021 04:06PM",87.278,10.29,NaN
4,3366880,Hermanus,ZA,19.23,-34.42,0.0,88.0,"Saturday, 13. February 2021 04:06PM",68.270,2.85,Misty Waves Boutique Hotel


In [65]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [67]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))